# Eksperimen Backtest Strategi Harian

Notebook ringkas untuk menjalankan satu strategi harian menggunakan pipeline terpusat.


In [1]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import pandas as pd
from IPython.display import display


def locate_project_root() -> Path:
    current = Path.cwd().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "src").exists():
            return candidate
    raise RuntimeError("Folder 'src' tidak ditemukan dari jalur kerja sekarang.")


PROJECT_ROOT = locate_project_root()
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

from src.pipelines.single_asset import (
    IndicatorConfig,
    SingleAssetPipelineConfig,
    run_single_asset_pipeline,
    save_backtest_outputs,
)

ModuleNotFoundError: No module named 'src'

In [ ]:
CONFIG = {
    "data_path": PROJECT_ROOT / "data/OKX_ETHUSDT.P, 60.csv",
    "strategy_name": "ema112",
    "strategy_kwargs": {},
    "price_column": "close",
    "horizon_bars": null,
    "indicators": [],
    "output_dir": PROJECT_ROOT / "outputs",
    "output_prefix": "backtest_strategy",
    "save_outputs": true,
}

PIPELINE_CONFIG = SingleAssetPipelineConfig(
    data_path=str(CONFIG['data_path']),
    strategy_name=CONFIG['strategy_name'],
    strategy_kwargs=CONFIG.get('strategy_kwargs', {}),
    horizon_bars=CONFIG.get('horizon_bars'),
    indicators=tuple(IndicatorConfig(**item) for item in CONFIG.get('indicators', [])),
    price_column=CONFIG.get('price_column', 'close'),
)

OUTPUT_DIR = CONFIG.get('output_dir', PROJECT_ROOT / 'outputs')
OUTPUT_PREFIX = CONFIG.get('output_prefix', 'backtest')
SAVE_OUTPUTS = bool(CONFIG.get('save_outputs', False))

In [ ]:
outputs = run_single_asset_pipeline(PIPELINE_CONFIG)

metrics_df = pd.Series(outputs.metrics, name="value").to_frame()
trade_summary_df = pd.Series(outputs.trade_summary, name="value").to_frame()

print("Ringkasan metrics:")
display(metrics_df.style.format({"value": "{:.4f}"}))

print("Ringkasan trade:")
display(trade_summary_df.style.format({"value": "{:.4f}"}))

print(f"Total trades: {outputs.trade_summary.get('total_trades', 0)}")
print(f"Sharpe Ratio: {outputs.metrics.get('sharpe_ratio'):.4f}")

if SAVE_OUTPUTS:
    artifacts = save_backtest_outputs(outputs, OUTPUT_DIR, prefix=OUTPUT_PREFIX)
    print("Artifacts tersimpan:")
    for name, path in artifacts.items():
        print(f"  - {name}: {path}")